In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
# Load both files
filenames = ["PrisonSampled2024.csv", "PrisonSampled2025.csv"]
combined_df = pd.concat([pd.read_csv(f) for f in filenames], ignore_index=True)

print(combined_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 44 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Entity.Code                   50000 non-null  object 
 1   Entity.Description            50000 non-null  object 
 2   NIGP..                        49999 non-null  float64
 3   NIGP.Description              49999 non-null  object 
 4   Item.Description              49792 non-null  object 
 5   Order..                       50000 non-null  object 
 6   Order.Line.Number             50000 non-null  int64  
 7   Quantity.Ordered              50000 non-null  float64
 8   Unit.Price                    50000 non-null  float64
 9   Unit.of.Measure               50000 non-null  object 
 10  Line.Total                    50000 non-null  float64
 11  Line.Total.Change             50000 non-null  float64
 12  Order.Status                  50000 non-null  object 
 13  S

In [ ]:
#Define the features we'll use and our target
features = [
    'Ordered.Date',
    'NIGP.Description',
    'Quantity.Ordered',
    'Vendor.Name'
]
target = 'Line.Total'

In [ ]:
# Create a new DataFrame from 'combined_df'
df = combined_df[features + [target]].copy()

In [ ]:
# Convert 'Ordered.Date' to datatime objects
df['Ordered.Date'] = pd.to_datetime(df['Ordered.Date'], errors='coerce')

Handle missing values

In [ ]:
# Drop any rows where we're missing the date or the price
df = df.dropna(subset=['Ordered.Date', 'Line.Total'])

In [ ]:
# Fill missing categorical data with a placeholder 'Missing'
df['NIGP.Description'] = df['NIGP.Description'].fillna('Missing')
df['Vendor.Name'] = df['Vendor.Name'].fillna('Missing')
df['Quantity.Ordered'] = df['Quantity.Ordered'].fillna(0)

Feature Creation (Time of Year)

In [ ]:
df['Order.Month'] = df['Ordered.Date'].dt.month
df['Order.Quarter'] = df['Ordered.Date'].dt.quarter
df['Order.Year'] = df['Ordered.Date'].dt.year

Encode Catagorical Data

In [ ]:
categorical_cols = ['NIGP.Description', 'Vendor.Name']
# Use OrdinalEncoder to covert them to numbers
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
# Store the original categorical data before encoding
original_categorical_data = df[categorical_cols].copy()
df[categorical_cols] = encoder.fit_transform(df[categorical_cols])

Build the Decision Tree Model

In [ ]:
# Define our final list of features for the model
model_features = ['Order.Month', 'Order.Quarter', 'Order.Year', 'NIGP.Description', 'Quantity.Ordered', 'Vendor.Name']
X = df[model_features]
y = df[target]

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize and train the model
# max_depth=10 prevents the tree from getting too big and overfitting
dt_model = DecisionTreeRegressor(max_depth=10, random_state=42)
dt_model.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=10, random_state=42)

Evaluate and Interpret the Model

In [ ]:
# Make predictions on the test set
y_pred = dt_model.predict(X_test)

In [ ]:
# See how accurate the model is
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"\n--- Model Results ---")
print(f"Model R-squared: {r2:.2f}")
print(f"Model Mean Absolute Error: ${mae:.2f}")
print(f"This means the model's cost predictions are off by an average of ${mae:.2f}.")


--- Model Results ---
Model R-squared: 0.10
Model Mean Absolute Error: $1226.66
This means the model's cost predictions are off by an average of $1226.66.


In [ ]:
print(f"\n--- Key Drivers of Procurement Cost (Feature Importances) ---")
# Get the importance of each feature
importances = dt_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': model_features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
print(feature_importance_df)


--- Key Drivers of Procurement Cost (Feature Importances) ---
            Feature  Importance
3  NIGP.Description    0.402771
5       Vendor.Name    0.363177
4  Quantity.Ordered    0.221662
0       Order.Month    0.011312
2        Order.Year    0.000854
1     Order.Quarter    0.000224


In [ ]:
# Create a copy of the DataFrame to add decoded columns
df_decoded = df.copy()

# Prepare the data for inverse transformation.
# The inverse_transform method expects input with the same number of features as the original fit.
# We need to provide both 'NIGP.Description' and 'Vendor.Name' columns.
encoded_data_for_decoding = df_decoded[categorical_cols].values

# Use the fitted encoder to inverse transform the numbers back into words
decoded_data = encoder.inverse_transform(encoded_data_for_decoding)

# Add the decoded columns to the new DataFrame
df_decoded['NIGP.Description_Decoded'] = decoded_data[:, categorical_cols.index('NIGP.Description')]
df_decoded['Vendor.Name_Decoded'] = decoded_data[:, categorical_cols.index('Vendor.Name')]

print("DataFrame with decoded columns:")
display(df_decoded.head())

DataFrame with decoded columns:


,Ordered.Date,NIGP.Description,Quantity.Ordered,Vendor.Name,Line.Total,Order.Month,Order.Quarter,Order.Year,NIGP.Description_Decoded,Vendor.Name_Decoded
0,2021-04-16,514.0,100.0,181.0,21.44,4,2,2021,"Washers, Steel",RE REPLACEMENT PARTS
7,2021-06-04,47.0,2800.0,28.0,62.72,6,2,2021,Bond Paper (Including Recycled),B W WILSON PAPER CO INC
8,2020-07-23,198.0,30.0,31.0,7084.50,7,3,2020,Furnishings (Not Otherwise Classified),BAY PRODUCT DEVELOPMENT LLC
9,2021-01-11,512.0,2.0,20.0,64.00,1,1,2021,WELDING EQUIPMENT AND SUPPLIES,ARC3 GASES
10,2021-04-13,357.0,10.0,231.0,24.90,4,2,2021,"Pens (General Writing Types): Ball Point, Nylo...",VA INDUSTRIES F T BLIND


In [ ]:
# Group by item description (decoded), sum the cost, and get the top 10 using df_decoded
top_10_categories = df_decoded.groupby('NIGP.Description_Decoded')['Line.Total'].sum().nlargest(10)

print("--- Top 10 Most Expensive Categories ---")
# Format the output with a dollar sign
top_10_categories = top_10_categories.apply(lambda x: f"${x:,.2f}")
print(top_10_categories)
top_10_category_list = top_10_categories.index # Save just the names for filtering

--- Top 10 Most Expensive Categories ---
NIGP.Description_Decoded
Signs, Electronic Display (Marquee, etc.)                     $874,270.81
Cloth (Yard Goods): Cotton                                    $730,788.15
Courier/Delivery Services (Including Air Courier Services)    $450,150.00
OFFICE MACHINES, EQUIPMENT, AND ACCESSORIES                   $409,016.92
Coil Stock                                                    $403,759.43
Transportation of Goods and Other Freight Services            $361,625.85
Consulting Services (Not Otherwise Classified)                $360,000.00
Household Furniture, General Line                             $351,561.13
Personnel Services, Temporary                                 $332,315.79
Lumber, Hardwood, All Kinds                                   $307,301.36
Name: Line.Total, dtype: object


In [ ]:
# Group by vendor (decoded), sum the cost, and get the top 10 using df_decoded
top_10_vendors_by_cost = df_decoded.groupby('Vendor.Name_Decoded')['Line.Total'].sum().nlargest(10)

print("\n--- Overall Top 10 Vendors by Total Spend ---")
# Format the output with a dollar sign
top_10_vendors_by_cost = top_10_vendors_by_cost.apply(lambda x: f"${x:,.2f}")
print(top_10_vendors_by_cost)
top_10_vendors_by_cost_list = top_10_vendors_by_cost.index # Save just the names for filtering


--- Overall Top 10 Vendors by Total Spend ---
Vendor.Name_Decoded
3M COMPANY                     $863,979.25
B W WILSON PAPER CO INC        $546,859.66
UPS MAIL INNOVATIONS           $450,000.00
TURBON USA INC                 $417,919.02
JUPITER ALUMINUM CORP          $399,613.24
INK2WORK LLC                   $360,000.00
BAY PRODUCT DEVELOPMENT LLC    $347,674.28
RICHELIEU HARDWARE             $339,726.03
COTSWOLD INDUSTRIES INC        $330,716.99
NIGHTINGALE CORP               $287,366.35
Name: Line.Total, dtype: object


In [ ]:
# Group by vendor, sum the quantity, and get the top 10
top_10_vendors_by_quantity = df_decoded.groupby('Vendor.Name_Decoded')['Quantity.Ordered'].sum().nlargest(10)

print("\n--- Top 10 Vendors by Total Quantity ---")
print(top_10_vendors_by_quantity)
top_10_vendors_by_quantity_list = top_10_vendors_by_quantity.index # Save just the names for filtering


--- Top 10 Vendors by Total Quantity ---
Vendor.Name_Decoded
B W WILSON PAPER CO INC    20749250.00
BOXWOOD SALES LLC           4289230.00
WALKER SUPPLY COMPANY       1555200.00
D & L SUPPLY INC             944640.00
GRAPHIC SIGN WORX LLC        864000.00
JUPITER ALUMINUM CORP        559222.64
3M COMPANY                   549600.00
TRIDENT PLASTICS INC         460800.00
UPS MAIL INNOVATIONS         450000.00
RICHELIEU HARDWARE           431663.00
Name: Quantity.Ordered, dtype: float64


In [ ]:
# Filter the DataFrame to *only* include rows for our top 10 categories using the decoded DataFrame
df_top_categories = df_decoded[df_decoded['NIGP.Description_Decoded'].isin(top_10_category_list)].copy()

# Now, group this smaller DataFrame by vendor (decoded name) to see who sells us these expensive items
vendors_in_top_categories = df_top_categories.groupby('Vendor.Name_Decoded')['Line.Total'].sum().sort_values(ascending=False)

print(f"\n--- Top Vendors for the {len(top_10_category_list)} Most Expensive Categories ---")

# Format the output with a dollar sign
vendors_in_top_categories = vendors_in_top_categories.apply(lambda x: f"${x:,.2f}")
print(vendors_in_top_categories.head(10)) # Show the top 10 from this new list


--- Top Vendors for the 10 Most Expensive Categories ---
Vendor.Name_Decoded
3M COMPANY                 $863,979.25
UPS MAIL INNOVATIONS       $450,000.00
TURBON USA INC             $407,919.02
JUPITER ALUMINUM CORP      $399,613.24
INK2WORK LLC               $360,000.00
COTSWOLD INDUSTRIES INC    $290,796.95
NIGHTINGALE CORP           $269,854.35
RIVERSIDE LOGISTICS        $250,000.00
COMPUTER AID INC           $225,201.60
RICHELIEU HARDWARE         $176,816.82
Name: Line.Total, dtype: object
